# 如何向Agent添加记忆

我们向智能体（Agent）添加了记忆组件。具体步骤如下：

首先，我们导入了所需的模块和类，包括 `ZeroShotAgent`、`Tool`、`AgentExecutor`、`ConversationBufferMemory`、`ChatOpenAI`、`LLMChain` 。



In [111]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory


使用 OpenAI，需要设置 openai_api_key 密钥，最好是使用 Chat Model 类的模型包装器 ChatOpenAI。

In [1]:
from langchain.chat_models import ChatOpenAI
openai_api_key="填入你的OPENAI_API_KEY密钥"
llm = ChatOpenAI(openai_api_key=openai_api_key,temperature=0, model="gpt-3.5-turbo-0613")
from langchain.chains import LLMChain

C:\Users\freestone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.19) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(



使用 Google 搜索，需要设置 SERPAPI_API_KEY 密钥。

In [106]:
import os
os.environ["SERPAPI_API_KEY"] = "填入你的SERPAPI_API_KEY密钥"

设置代理类型为 ZERO_SHOT_REACT_DESCRIPTION，加载所有的 tools，初始化这个代理。运行代理，我们询问“请告诉我OPENAI的CEO是谁？”。

In [105]:
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,

)
agent.run("请告诉我OPENAI的CEO是谁？")

'OPENAI的CEO是Sam Altman。'

现在开始加入记忆组件。`Tools` 对象将作为智能体的一个工具，用于回答有关当前事件的问题可以借助的工具。我们指定了 `LLMChain` 作为链，并设置了 `tools` 作为工具。

接着，我们创建了一个 `LLMChain` 实例，并使用它来初始化 `ZeroShotAgent`。在 `LLMChain` 的初始化中，我们指定了 `ChatOpenAI` 作为大语言模型，并设置了 `prompt` 作为提示词模板。在 `ZeroShotAgent` 的初始化中，我们指定了 `LLMChain` 作为链，并设置了 `tools` 作为工具。

然后，我们将 `ZeroShotAgent` 和 `tools` 一起传入 `AgentExecutor.from_agent_and_tools` 方法，创建了一个 `AgentExecutor` 实例。在这个过程中，我们还指定了 `memory` 作为记忆组件。

最后，我们使用 `AgentExecutor` 实例的 `run` 方法，运行智能体，并向其提出问题 "上海的人口是多少？"。



In [112]:
prefix = """你是一个说中文的chatbot,你可以使用tools帮你获得答案:"""
suffix = """你的中文回答是："

聊天记录：{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

agent_chain.run("上海的人口是多少？")






> Entering new AgentExecutor chain...
Thought: 我需要搜索一下上海的人口数据。
Action: Search
Action Input: "上海人口数据"
Observation: 26.32 million (2019)
Thought:我现在知道了上海的人口是2632万（2019年）。
Final Answer: 上海的人口是2632万（2019年）。

> Finished chain.


'上海的人口是2632万（2019年）。'

为了测试加入记忆组件的 Agent 是否更加智能，第二个问题，使用代词“它”来迷惑大语言模型，就像我们跟朋友聊天时，我们谈论一个人，可能只会提及一次姓名。后面的聊天都会使用人称代词或者缩写，而不用每次都使用全名。

In [113]:
agent_chain.run("它的地标建筑是什么？")



> Entering new AgentExecutor chain...
Thought: 我需要搜索上海的地标建筑。
Action: Search
Action Input: 上海地标建筑
Observation: https://cn.tripadvisor.com/Attractions-g308272-Activities-c47-Shanghai.html
Thought:我现在知道上海的地标建筑了。
Final Answer: 上海的地标建筑包括东方明珠广播电视塔、外滩、上海博物馆等。

> Finished chain.


'上海的地标建筑包括东方明珠广播电视塔、外滩、上海博物馆等。'

我们可以看到，虽然代码的大部分步骤都和之前一样，但是在创建 `AgentExecutor` 的时候，我们并没有指定 `memory` 参数。

为了对比，记忆组件在Agent组件中，发挥的什么作用？我们再创建一个Agent组件，但是不要加入记忆组件。我们使用相同的问题，测试Agent是否知道第二个问题“它的地标建筑是什么”，“它”指代的是“上海”。

In [109]:
prefix = """你是一个说中文的chatbot,你可以使用tools帮你获得答案:"""
suffix = """Begin!"


Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

agent_chain.run("上海的人口是多少？")



> Entering new AgentExecutor chain...
Thought: I need to find the current population of Shanghai.
Action: Search
Action Input: "上海的人口是多少？"
Observation: 26.32 million (2019)
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-0WG0lwoFORBdfkEYprKFZ7FW on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-0WG0lwoFORBdfkEYprKFZ7FW on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I now know the current population of Shanghai.
Final Answer: The population of Shanghai is 26.32 million (2019).

> Finished chain.


'The population of Shanghai is 26.32 million (2019).'

从给出的答案“ 'The landmark building of "it" is the Empire State Building in New York City.' ”,我们可以对比得出，没有记忆组件的 Agent,面对“它”这样的指示代词无能无力，给出了莫名其妙的一个答案。它并不能关联上下文，推理出“它”指的是“上海”。

In [110]:
agent_chain.run("它的地标建筑是什么？")



> Entering new AgentExecutor chain...
Thought: I need to find out the landmark building of "it".
Action: Search
Action Input: "it landmark building"
Observation: Landmark Builds, Joplin, Missouri. 186 likes · 1 talking about this. Engaging teens in local history and STEM education by tapping the creative power of ...
Thought:This search result is not relevant to the question. I need to refine my search query.
Action: Search
Action Input: "it city landmark building"
Observation: Landmark buildings are icons of a place. They create a statement about the city's legacy and influence how we think of that place. Landmark buildings stand out ...
Thought:This search result is still not relevant to the question. I need to refine my search query further.
Action: Search
Action Input: "it city famous landmark building"
Observation: Empire State Building, Manhattan, New York City, USA, the Art Deco skyscraper is New York's most popular landmark and a symbol for the American way of life, ...
Thoug

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-0WG0lwoFORBdfkEYprKFZ7FW on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-0WG0lwoFORBdfkEYprKFZ7FW on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

This search result is relevant to the question. The landmark building of "it" is the Empire State Building in New York City.
Thought: I now know the final answer.
Final Answer: The landmark building of "it" is the Empire State Building in New York City.

> Finished chain.


'The landmark building of "it" is the Empire State Building in New York City.'

总的来说，这段代码展示了如何向 Agent 添加记忆组件，以及记忆组件在 Agent 中的作用。